In [73]:
# Load Packages
from __future__ import unicode_literals, print_function

import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm, tqdm_notebook # loading bar 
import pandas as pd
import re
from pprint import pprint
from nltk.tokenize import sent_tokenize
import numpy as np

from spacy.util import minibatch, compounding

# Abrindo e gerando Datasets

### Entidade vazia

In [43]:
# Abrindo o Dataset de Corpus sem entidades
# Gerando um Dataframe
with open('corpus_paramopama+second_harem.txt') as f:
    to_df = []
    
    text = ''
    for line in f:
        if(line.split('\t')[0] == '.'):
            text = text.replace('\n ','')
            text = text.lower()
            to_df.append(text)
            
#             print('text:', text)
            text = ''
        else:
            
            splitted = line.split('\t')[0]+' '
            text += splitted
#             print('append', splitted)

df = pd.DataFrame({0:to_df})

In [44]:
df[:10]

,0
0,"a bandeira nacional , adotada em 1947 , é base..."
1,jana gana mana é o hino nacional da índia
2,"com um pib de 785 bilhões de dólares ( ou 3,6 ..."
3,"entretanto , devido à grande população , a ren..."
4,a índia registrou forte crescimento econômico ...
5,"nos últimos anos , a índia tornou-se um import..."
6,a índia também é a maior produtora de software...
7,a cultura da índia é a expressão de uma das ma...
8,as tradições literárias mais antigas da índia ...
9,"no entando , o programa microsoft excel consid..."


In [45]:
# Obtendo dados sem entidade

# Obtendo endereços do DataFrame
end_lista = []
for i in range(len(df)):
    str_raw = df[0][i]
    str_tratada = re.sub(r'[ ]{2,}', "",str_raw) # Tirando espaços excedentes no final do endereço
    end_lista.append(str_tratada)

# Extraindo iob
iob = []
for i in end_lista:  
    # Dicionário contando entidade (somente entidade endereço END)
    iob_dict = {"entities": []}   
    
    tupla = (i, iob_dict)
    iob.append(tupla)


TRAIN_DATA_No_ents = iob
len(TRAIN_DATA_No_ents)

15693

In [46]:
print(TRAIN_DATA_No_ents[:10])

[('a bandeira nacional , adotada em 1947 , é baseada na bandeira do congresso nacional indiano , desenhada por pingali venkayya ', {'entities': []}), ('jana gana mana é o hino nacional da índia ', {'entities': []}), ('com um pib de 785 bilhões de dólares ( ou 3,6 trilhões de dólares pelo critério de paridade do poder de compra -- ppc ) , a índia é a 12ª maior economia do mundo ( ou a quarta maior , pelo critério ppc ) ', {'entities': []}), ('entretanto , devido à grande população , a renda per capita é consideravelmente baixa : em 2005 , o fmi classificou a índia na 135ª posição em termos de renda per capita ( ou na 122ª posição , pelo critério ppc ) , dentre 182 países e territórios ', {'entities': []}), ('a índia registrou forte crescimento econômico após 1991 , quando seu governo abandonou políticas socialistas e deu início a um processo de liberalização da economia , que envolveu o incentivo ao investimento estrangeiro , a redução de barreiras tarifárias à importação , a modernizaç

### Entidade endereço

In [59]:
# Obtendo dados com entidades

dataset = pd.read_csv("201906AGENCIAS.CSV")
dset = dataset.iloc[:-2,4:10].values  # tipo nparray object
pd_dset = pd.DataFrame(dset) # caso queira visualizar

pd_dset['Join'] = pd_dset[pd_dset.columns[0:]].apply(
    lambda x: ';'.join(x.dropna().astype(str)),
    axis=1
)
pd_dset.head(5)

,0,1,2,3,4,5,Join
0,"R.GUILHERME MOREIRA,315","SUBLOJA,LOJA E 2.ANDAR ...",CENTRO,69005-300,MANAUS ...,AM,"R.GUILHERME MOREIRA,315 ;SUBLO..."
1,"AV.PRES.VARGAS,248",1.E 2.ANDARES ...,CAMPINA,66010-900,BELEM ...,PA,"AV.PRES.VARGAS,248 ;1.E 2..."
2,"R.QUINZE DE NOVEMBRO,195",...,CENTRO,11010-908,SANTOS ...,SP,"R.QUINZE DE NOVEMBRO,195 ; ..."
3,"PCA.DAS QUATRO JORNADAS,11",MEZANINO ...,CENTRO,28010-000,CAMPOS DOS GOYTACAZES ...,RJ,"PCA.DAS QUATRO JORNADAS,11 ;MEZAN..."
4,"SEXTA AVENIDA,600",SECRETARIA DA EDUCACAO-TERREO ...,CAB,41745-002,SALVADOR ...,BA,"SEXTA AVENIDA,600 ;SECRE..."


In [51]:
pd_dset["Join"][:10]

0    R.GUILHERME MOREIRA,315                 ;SUBLO...
1    AV.PRES.VARGAS,248                      ;1.E 2...
2    R.QUINZE DE NOVEMBRO,195                ;     ...
3    PCA.DAS QUATRO JORNADAS,11              ;MEZAN...
4    SEXTA AVENIDA,600                       ;SECRE...
5    AV.RIO BRANCO,240                       ;1.AND...
6    AV.SANTOS DUMONT,2828                   ;5.AND...
7    PCA.TIRADENTES,410                      ;1.AND...
8    R.URUGUAI,185                           ;5.AND...
9    PCA.1817,129                            ;1.AND...
Name: Join, dtype: object

In [142]:
# Obtendo endereços do DataFrame
dset = np.array(pd_dset)
end_lista = []

for i in range(len(dset)):
    str_raw = dset[i][6]
    str_tratada = re.sub(r'[ ]{2,}', "",str_raw) # Tirando espaços excedentes no final do endereço
    str_tratada = str_tratada.lower()
    
    end_lista.append(str_tratada)
 
# Extraindo iob
iob = []
for i in end_lista:  
    # Dicionário contando entidade (somente entidade endereço END)
    iob_dict = {"entities": [(0,len(i)-1,'END')]}   
    
    tupla = (i, iob_dict)
    iob.append(tupla)


TRAIN_DATA_ents = iob
print(len(TRAIN_DATA_ents))

21336


In [143]:
TRAIN_DATA_ents[0]

('r.guilherme moreira,315;subloja,loja e 2.andar;centro;69005-300;manaus;am',
 {'entities': [(0, 72, 'END')]})

In [8]:
# Concatenando os dois datasets

Full_Frame = TRAIN_DATA_ents + TRAIN_DATA_No_ents
random.shuffle(Full_Frame)

Full_Frame[:5]

[('ruacel joao camilo', {'entities': [(0, 17, 'END')]}),
 ('o sindicato contesta que não tenha apoiado os funcionários ',
  {'entities': []}),
 ('hoje a economia da terra chã é dominada de forma esmagadora pelo trabalho assalariado na zona urbana de angra do heroísmo , estando em rápido desaparecimentos os últimos traços de ruralidade que ainda subsistem ',
  {'entities': []}),
 ('as características climáticas da região são condicionadas pelas diferenças de altitudes , que vão desde o nível do mar até 4110m , no monte tajumulco ',
  {'entities': []}),
 ('mesmo com o mundial , só iniciei a preparação em julho , enquanto os colegas começaram em maio , e isso notou - se » , explicou com alguma mágoa ',
  {'entities': []}),
 ('avenida getulio vargas, 510', {'entities': [(0, 26, 'END')]}),
 ('em novembro de 1918 , a hungria tornou-se uma república independente ',
  {'entities': []}),
 ('av.dom pedro primeiro,877', {'entities': [(0, 24, 'END')]}),
 ('r.vol.da patria,1456/1458', {'entities': 

In [9]:
# Criação da base de teste e treinamento
n_test= 0.1 # Porcentagem para base de teste
test_n = round(len(Full_Frame) * n_test)

# Divisao em Train Test Val

def gerador_bases(dataset, n):
    indices_random = random.sample(range(0,len(dataset)-1), n)
    base_teste_n = []
    base_treinamento_n = []
    for i in range(n):
        base_teste_n.append(dataset[indices_random[i]])
#     print(indices_random)
    for j in range(len(dataset)):
        if(j not in indices_random):
            base_treinamento_n.append(dataset[j])
            
    return base_teste_n, base_treinamento_n

base_teste_final, base_treinamento_final = gerador_bases(Full_Frame, test_n)
print("Treinamento: " + str(len(base_treinamento_final)), "\nTeste: " + str(len(base_teste_final)), "\nTotal: " + str(len(Full_Frame)))

Treinamento: 28196 
Teste: 3133 
Total: 31329


In [10]:
base_treinamento_final

[('ruacel joao camilo', {'entities': [(0, 17, 'END')]}),
 ('o sindicato contesta que não tenha apoiado os funcionários ',
  {'entities': []}),
 ('hoje a economia da terra chã é dominada de forma esmagadora pelo trabalho assalariado na zona urbana de angra do heroísmo , estando em rápido desaparecimentos os últimos traços de ruralidade que ainda subsistem ',
  {'entities': []}),
 ('as características climáticas da região são condicionadas pelas diferenças de altitudes , que vão desde o nível do mar até 4110m , no monte tajumulco ',
  {'entities': []}),
 ('mesmo com o mundial , só iniciei a preparação em julho , enquanto os colegas começaram em maio , e isso notou - se » , explicou com alguma mágoa ',
  {'entities': []}),
 ('avenida getulio vargas, 510', {'entities': [(0, 26, 'END')]}),
 ('av.dom pedro primeiro,877', {'entities': [(0, 24, 'END')]}),
 ('r.vol.da patria,1456/1458', {'entities': [(0, 24, 'END')]}),
 ('av dr mendel steinbruch, s/n', {'entities': [(0, 27, 'END')]}),
 ('ele é 

# Carregando o modelo

In [11]:
# nlp1 = spacy.load('pt_core_news_sm') # desnecessário?

In [12]:
# Define our variables
model = None
# model = 'pt_core_news_sm'
output_dir=Path(".")
n_iter= 10 # número de épocas

In [13]:
# Setting up the pipeline and entity recognizer.
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    lan = 'pt'
    nlp = spacy.blank(lan)  # create blank Language class
    print("Created blank '%s' model" % lan)
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
    print('Added new NER')
else:
    ner = nlp.get_pipe('ner')
    print('Got an old NER')

Created blank 'pt' model
Added new NER


In [14]:
# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

## Treinamento

In [15]:
# add labels
for _, annotations in base_treinamento_final:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

    # get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(base_treinamento_final)
        losses = {}
        batches = minibatch(Full_Frame, size=128)
        
        for batch in batches:
            texts, annotations = zip(*batch)
            try:
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
            except:
                pass     
        
        print(itn+1, ' Losses', losses)        

0  Losses {'ner': 7696.310477406831}
1  Losses {'ner': 187.26279561646723}
2  Losses {'ner': 134.40938916325672}
3  Losses {'ner': 143.20132559269774}
4  Losses {'ner': 61.710469198457545}
5  Losses {'ner': 100.50243327169042}
6  Losses {'ner': 85.72868148333863}
7  Losses {'ner': 55.534096637171835}
8  Losses {'ner': 51.40160562127562}
9  Losses {'ner': 36.95210439779156}


In [16]:
print(losses)

{'ner': 36.95210439779156}


In [17]:
# test the trained model
for text, _ in base_teste_final:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    print()

Entities []
Tokens [('este', '', 2), ('levantamento', '', 2), ('de', '', 2), ('preços', '', 2), ('que', '', 2), ('inclui', '', 2), ('também', '', 2), ('produtos', '', 2), ('de', '', 2), ('higiene', '', 2), ('e', '', 2), ('de', '', 2), ('limpeza', '', 2), ('mostra', '', 2), ('recuo', '', 2), ('de', '', 2), ('1,58', '', 2), ('%', '', 2), ('em', '', 2), ('relação', '', 2), ('à', '', 2), ('sexta-feira', '', 2), ('anterior', '', 2)]

Entities []
Tokens [('marx', '', 2), ('procurou', '', 2), ('demonstrar', '', 2), ('a', '', 2), ('dinâmica', '', 2), ('econômica', '', 2), ('que', '', 2), ('levou', '', 2), ('a', '', 2), ('sociedade', '', 2), (',', '', 2), ('partindo', '', 2), ('do', '', 2), ('comunismo', '', 2), ('primitivo', '', 2), (',', '', 2), ('até', '', 2), ('a', '', 2), ('concentração', '', 2), ('cada', '', 2), ('vez', '', 2), ('mais', '', 2), ('acentuada', '', 2), ('do', '', 2), ('capital', '', 2), ('e', '', 2), ('o', '', 2), ('aparecimento', '', 2), ('da', '', 2), ('classe', '', 2), ('

Entities []
Tokens [('a', '', 2), ('bulgária', '', 2), ('é', '', 2), ('um', '', 2), ('país', '', 2), ('dos', '', 2), ('balcãs', '', 2), (',', '', 2), ('limitado', '', 2), ('a', '', 2), ('norte', '', 2), ('pela', '', 2), ('roménia', '', 2), (',', '', 2), ('a', '', 2), ('leste', '', 2), ('pelo', '', 2), ('mar', '', 2), ('negro', '', 2), (',', '', 2), ('a', '', 2), ('sul', '', 2), ('pela', '', 2), ('turquia', '', 2), ('e', '', 2), ('pela', '', 2), ('grécia', '', 2), ('e', '', 2), ('a', '', 2), ('oeste', '', 2), ('pela', '', 2), ('macedónia', '', 2), ('e', '', 2), ('pela', '', 2), ('sérvia', '', 2)]

Entities []
Tokens [('na', '', 2), ('floresta', '', 2), ('da', '', 2), ('planície', '', 2), ('de', '', 2), ('el', '', 2), ('petén', '', 2), ('o', '', 2), ('clima', '', 2), ('é', '', 2), ('quente', '', 2), (',', '', 2), ('e', '', 2), ('o', '', 2), ('grau', '', 2), ('de', '', 2), ('umidade', '', 2), ('varia', '', 2), ('de', '', 2), ('acordo', '', 2), ('com', '', 2), ('a', '', 2), ('época', '', 2

Entities []
Tokens [('av.dos', '', 2), ('ipes', '', 2)]

Entities []
Tokens [('seu', '', 2), ('envolvimento', '', 2), ('com', '', 2), ('as', '', 2), ('máquinas', '', 2), ('é', '', 2), ('tão', '', 2), ('intenso', '', 2), (',', '', 2), ('tão', '', 2), ('concentrado', '', 2), ('que', '', 2), (',', '', 2), ('como', '', 2), ('ele', '', 2), ('se', '', 2), ('lança', '', 2), ('do', '', 2), ('mixer', '', 2), ('para', '', 2), ('a', '', 2), ('picape', '', 2), (',', '', 2), ('mills', '', 2), ('o', '', 2), ('dj', '', 2), ('parece', '', 2), ('evidentemente', '', 2), ('um', '', 2), ('componente', '', 2), ('de', '', 2), ('uma', '', 2), ('assemblage', '', 2), ('homem', '', 2), ('-', '', 2), ('máquina', '', 2), (',', '', 2), ('um', '', 2), ('sistema', '', 2), ('que', '', 2), ('inclui', '', 2), ('público', '', 2), (',', '', 2), ('pa', '', 2), (',', '', 2), ('o', '', 2), ('aparato', '', 2), ('completo', '', 2), ('da', '', 2), ('produção', '', 2), ('de', '', 2), ('vinis', '', 2), (',', '', 2), ('e', '', 2)

Entities [('av.antonelo da messina, 1175', 'END')]
Tokens [('av.antonelo', 'END', 3), ('da', 'END', 1), ('messina', 'END', 1), (',', 'END', 1), ('1175', 'END', 1)]

Entities []
Tokens [('é', '', 2), ('de', '', 2), ('competência', '', 2), ('exclusiva', '', 2), ('do', '', 2), ('banco', '', 2), ('central', '', 2), ('do', '', 2), ('brasil', '', 2), (':', '', 2), ('surgiu', '', 2), ('quando', '', 2), ('evangelista', '', 2), ('torricelli', '', 2), ('inventou', '', 2), ('o', '', 2), ('barómetro', '', 2), ('de', '', 2), ('mercúrio', '', 2), (',', '', 2), ('em', '', 2), ('1643', '', 2), ('e', '', 2), ('tem', '', 2), ('vindo', '', 2), ('a', '', 2), ('cair', '', 2), ('em', '', 2), ('desuso', '', 2), ('com', '', 2), ('o', '', 2), ('aparecimento', '', 2), ('de', '', 2), ('tecnologia', '', 2), ('mais', '', 2), ('eficaz', '', 2), ('para', '', 2), ('a', '', 2), ('medição', '', 2), ('da', '', 2), ('pressão', '', 2), ('atmosférica', '', 2), ('e', '', 2), ('com', '', 2), ('a', '', 2), ('disseminação', ''

Entities []
Tokens [('d', '', 2), ('esta', '', 2), ('forma', '', 2), ('foi', '', 2), ('eleito', '', 2), ('santiago', '', 2), ('derqui', '', 2), ('para', '', 2), ('presidente', '', 2), ('e', '', 2), ('urquiza', '', 2), ('e', '', 2), ('bartolomé', '', 2), ('mitre', '', 2), ('foram', '', 2), ('nomeados', '', 2), ('governadores', '', 2), ('de', '', 2), ('entre', '', 2), ('ríos', '', 2), ('e', '', 2), ('buenos', '', 2), ('aires', '', 2), (',', '', 2), ('respectivamente', '', 2)]

Entities [('av. fernando correa da costa', 'END'), ('2206', 'END')]
Tokens [('av', 'END', 3), ('.', 'END', 1), ('fernando', 'END', 1), ('correa', 'END', 1), ('da', 'END', 1), ('costa', 'END', 1), (',', '', 2), ('2206', 'END', 3)]

Entities []
Tokens [('inicialmente', '', 2), ('restrita', '', 2), ('a', '', 2), ('o', '', 2), ('círculo', '', 2), ('de', '', 2), ('programadores', '', 2), ('e', '', 2), ('geeks', '', 2), (',', '', 2), ('a', '', 2), ('idéia', '', 2), ('da', '', 2), ('criação', '', 2), ('coletiva', '', 2), 

...

Entities [('av. ataulfo de paiva, 270, lojas a e b', 'END')]
Tokens [('av', 'END', 3), ('.', 'END', 1), ('ataulfo', 'END', 1), ('de', 'END', 1), ('paiva', 'END', 1), (',', 'END', 1), ('270', 'END', 1), (',', 'END', 1), ('lojas', 'END', 1), ('a', 'END', 1), ('e', 'END', 1), ('b', 'END', 1)]

Entities []
Tokens [('a', '', 2), ('leitura', '', 2), ('de', '', 2), ('todo', '', 2), ('o', '', 2), ('trecho', '', 2), ('leva', '', 2), (',', '', 2), ('aliás', '', 2), (',', '', 2), ('a', '', 2), ('outra', '', 2), ('mentira', '', 2), (',', '', 2), ('esta', '', 2), ('sobre', '', 2), ('os', '', 2), ('tempos', '', 2), ('de', '', 2), ('tv', '', 2), ('do', '', 2), ('psdb', '', 2), ('e', '', 2), ('do', '', 2), ('pt', '', 2)]

Entities []
Tokens [('o', '', 2), ('atentado', '', 2), ('teria', '', 2), ('sido', '', 2), ('planejado', '', 2), ('pelo', '', 2), ('grupo', '', 2), ('radical', '', 2), ('islâmico', '', 2), ("gama'a", '', 2), ('al-islamiya', '', 2), (',', '', 2), ('que', '', 2), ('lançou', '', 2), ('ca

Entities [('rua marques de abrantes, 162', 'END')]
Tokens [('rua', 'END', 3), ('marques', 'END', 1), ('de', 'END', 1), ('abrantes', 'END', 1), (',', 'END', 1), ('162', 'END', 1)]

Entities []
Tokens [('além', '', 2), ('de', '', 2), ('brincar', '', 2), (',', '', 2), ('é', '', 2), ('uma', '', 2), ('experiência', '', 2)]

Entities []
Tokens [('r.felipe', '', 2), ('schmidt,312', '', 2)]

Entities [('av maurilio biagi 800 - 16� andar', 'END')]
Tokens [('av', 'END', 3), ('maurilio', 'END', 1), ('biagi', 'END', 1), ('800', 'END', 1), ('-', 'END', 1), ('16', 'END', 1), ('�', 'END', 1), ('andar', 'END', 1)]

Entities [('rua jose pires neto, 314', 'END')]
Tokens [('rua', 'END', 3), ('jose', 'END', 1), ('pires', 'END', 1), ('neto', 'END', 1), (',', 'END', 1), ('314', 'END', 1)]

Entities [('rua santa rita, n� 327', 'END')]
Tokens [('rua', 'END', 3), ('santa', 'END', 1), ('rita', 'END', 1), (',', 'END', 1), ('n', 'END', 1), ('�', 'END', 1), ('327', 'END', 1)]

Entities [('r da bahia,2500', 'END')]

Entities []
Tokens [('o', '', 2), ('senegal', '', 2), ('tornou-se', '', 2), ('uma', '', 2), ('república', '', 2), ('autônoma', '', 2), ('dentro', '', 2), ('da', '', 2), ('comunidade', '', 2), ('francesa', '', 2), ('em', '', 2), ('1958', '', 2)]

Entities []
Tokens [('srtvs', '', 2), (',', '', 2), ('qd.701,cj.d', '', 2), (',', '', 2), ('bloco', '', 2), ('b', '', 2), (',', '', 2), ('loja', '', 2), ('12', '', 2)]

Entities [('avenida afonso pena, 3504', 'END')]
Tokens [('avenida', 'END', 3), ('afonso', 'END', 1), ('pena', 'END', 1), (',', 'END', 1), ('3504', 'END', 1)]

Entities []
Tokens [('yakovlev', '', 2), ('vence', '', 2), ('maynard', '', 2), (',', '', 2), ('que', '', 2), ('chega', '', 2), ('a', '', 2), ('o', '', 2), ('quarto', '', 2), ('revés', '', 2), ('consecutivo', '', 2), ('no', '', 2), ('ufc', '', 2)]

Entities []
Tokens [('guatemala', '', 2), ('reconheceu', '', 2), ('a', '', 2), ('república', '', 2), ('em', '', 2), ('1992', '', 2)]

Entities [('rua frei gaspar, 493', 'END')]
T

Entities []
Tokens [('circunscrevendo', '', 2), ('-', '', 2), ('se', '', 2), (',', '', 2), ('pela', '', 2), ('facilidade', '', 2), ('de', '', 2), ('informações', '', 2), ('diretas', '', 2), (',', '', 2), ('a', '', 2), ('o', '', 2), ('que', '', 2), ('se', '', 2), ('vem', '', 2), ('fazendo', '', 2), ('no', '', 2), ('rio', '', 2), ('de', '', 2), ('janeiro', '', 2), (',', '', 2), ('cabe', '', 2), ('menção', '', 2), ('a', '', 2), ('um', '', 2), ('ou', '', 2), ('dois', '', 2), ('capítulos', '', 2), ('da', '', 2), ('1', '', 2)]

Entities [('av.cel colares', 'END')]
Tokens [('av.cel', 'END', 3), ('colares', 'END', 1), ('moreira,18', '', 2)]

Entities []
Tokens [('os', '', 2), ('encargos', '', 2), ('com', '', 2), ('as', '', 2), ('demissões', '', 2), ('serão', '', 2), ('contabilizados', '', 2), ('no', '', 2), ('resultado', '', 2), ('da', '', 2), ('oi', '', 2), ('do', '', 2), ('segundo', '', 2), ('trimestre', '', 2)]

Entities []
Tokens [('outro', '', 2), ('lado', '', 2), ('de', '', 2), ('lara', 

Entities [('rod.washington luiz,km', 'END')]
Tokens [('rod.washington', 'END', 3), ('luiz', 'END', 1), (',', 'END', 1), ('km', 'END', 1), ('113,7', '', 2)]

Entities []
Tokens [('em', '', 2), ('1816', '', 2), ('ele', '', 2), ('se', '', 2), ('reúne', '', 2), ('com', '', 2), ('representantes', '', 2), ('da', '', 2), ('burguesia', '', 2), ('para', '', 2), ('criar', '', 2), ('uma', '', 2), ('colônia', '', 2), ('na', '', 2), ('áfrica', '', 2)]

Entities [('rua afonso cavalcanti, 73-a', 'END')]
Tokens [('rua', 'END', 3), ('afonso', 'END', 1), ('cavalcanti', 'END', 1), (',', 'END', 1), ('73-a', 'END', 1)]

Entities []
Tokens [('av.parana,330', '', 2)]

Entities [('av. julio borella, 805', 'END')]
Tokens [('av', 'END', 3), ('.', 'END', 1), ('julio', 'END', 1), ('borella', 'END', 1), (',', 'END', 1), ('805', 'END', 1)]

Entities [('av. saul elkind, 1.394', 'END')]
Tokens [('av', 'END', 3), ('.', 'END', 1), ('saul', 'END', 1), ('elkind', 'END', 1), (',', 'END', 1), ('1.394', 'END', 1)]

Entities

In [36]:
base_teste_final[0]

('este levantamento de preços que inclui também produtos de higiene e de limpeza mostra recuo de 1,58 % em relação à sexta-feira anterior ',
 {'entities': []})

In [43]:
phrase = "rua leonardo de freitas, 1444,   este levantamento de preços que inclui também produtos de higiene e de limpeza mostra recuo de 1,58 % em relação à sexta-feira anterior \n "

doc = nlp(phrase)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('rua leonardo de freitas, 1444,   este levantamento de preços que inclui também produtos de higiene e de limpeza mostra recuo de 1,58 % em relação à sexta-feira anterior \n ', 'END')]
Tokens [('rua', 'END', 3), ('leonardo', 'END', 1), ('de', 'END', 1), ('freitas', 'END', 1), (',', 'END', 1), ('1444', 'END', 1), (',', 'END', 1), ('  ', 'END', 1), ('este', 'END', 1), ('levantamento', 'END', 1), ('de', 'END', 1), ('preços', 'END', 1), ('que', 'END', 1), ('inclui', 'END', 1), ('também', 'END', 1), ('produtos', 'END', 1), ('de', 'END', 1), ('higiene', 'END', 1), ('e', 'END', 1), ('de', 'END', 1), ('limpeza', 'END', 1), ('mostra', 'END', 1), ('recuo', 'END', 1), ('de', 'END', 1), ('1,58', 'END', 1), ('%', 'END', 1), ('em', 'END', 1), ('relação', 'END', 1), ('à', 'END', 1), ('sexta-feira', 'END', 1), ('anterior', 'END', 1), ('\n ', 'END', 1)]


In [27]:
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

In [25]:
# save model to output directory
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to .


In [28]:
loaded_model = spacy.load(output_dir)
evaluate(loaded_model, base_teste_final)

IndexError: list index out of range